In [2]:
import pandas as pd
import os
os.chdir(r'D:\PycharmProjects\pythonStudy\Pandas')

# 1. 演示分组对象

In [ ]:
# 1. 准备动作, 读取数据, 获取df对象.
df = pd.read_csv('./data/uniqlo.csv')
df

In [ ]:
# 2. 基于一列进行分组, 获得: DataFrameGroupBy分组对象.
# 需求: 基于 顾客的性别 分组.
df.groupby('gender_group')              # <pandas.core.groupby.generic.DataFrameGroupBy object at 0x0000025029082030>
df.groupby('gender_group')['city']      # <pandas.core.groupby.generic.SeriesGroupBy object at 0x000002502AC56150>

# 用变量来接收 分组对象, 获取某个分组的信息.
df_gb = df.groupby('gender_group')      # 根据性别分组.
print(df_gb.get_group('Female'))        # 根据 分组名 获取到该组的信息

print(df_gb['city'])

In [ ]:
# 3. 基于多列进行分组, 获得: DataFrameGroupBy分组对象.
# 需求: 基于 顾客的性别 和 城市 分组.
df.groupby(['gender_group', 'city'])            # <pandas.core.groupby.generic.DataFrameGroupBy object at 0x000002502900FC50>
df.groupby(['gender_group', 'city'])['revenue'] # <pandas.core.groupby.generic.SeriesGroupBy object at 0x000002502AC563F0>

In [ ]:
# 4. 分组后, 获取到每个分组的第一条 或者 最后一条数据.
# 组后第一条: first()
# 组后最后一条: last()

# 4.1 变量记录, 分组对象.
df_gb = df.groupby(['gender_group', 'city'])
df_gb.first()
df_gb.last()

# 2. 演示分组聚合操作

In [5]:
data = {
    'col1': ['A', 'B', 'A', 'B', 'A', 'C'],
    'col2': [10, 20, 30, 40, 50, 60]
}
dfl = pd.DataFrame(data)
dfl


,col1,col2
0,A,10
1,B,20
2,A,30
3,B,40
4,A,50
5,C,60


In [7]:
dfl['mean_col2'] = dfl.groupby('col1')['col2'].transform('mean')
dfl

,col1,col2,mean_col2
0,A,10,30.0
1,B,20,30.0
2,A,30,30.0
3,B,40,30.0
4,A,50,30.0
5,C,60,60.0


In [8]:
dfl['sum_col2']=dfl.groupby('col1')['col2'].transform('sum')
dfl

,col1,col2,mean_col2,sum_col2
0,A,10,30.0,90
1,B,20,30.0,60
2,A,30,30.0,90
3,B,40,30.0,60
4,A,50,30.0,90
5,C,60,60.0,60


In [ ]:
# 格式: df.groupby(['分组字段1', '字段2'...]).agg({'列名1':'聚合函数名', '列名2':'聚合函数名'...})
# 格式: df.pivot_table(index='行索引', columns='列', values='值', aggfunc='聚合函数名')

# 需求1: 按照城市分组, 计算每个城市的 客户数量. 
# 方式1: groupby() + 聚合函数.
df.groupby('city').customer.sum()
df.groupby('city').customer.agg('sum')      # 效果同上
df.groupby('city').agg({'customer':'sum'})  # 效果同上

# 方式2: pivot_table() 透视表
df.pivot_table(index='city', values='customer', aggfunc='sum')

In [ ]:
# 需求2: 按照城市, 性别分组, 计算每个城市的 客户数量. 
# 方式1: groupby() + 聚合函数.
df.groupby(['city', 'gender_group']).customer.sum()
df.groupby(['city', 'gender_group']).customer.agg('sum')
df.groupby(['city', 'gender_group']).agg({'customer': 'sum'})

# 方式2: pivot_table() 透视表
df.pivot_table(index=['city', 'gender_group'], values='customer', aggfunc='sum')

# 上述格式的变形写法, 类似于: 行列转置, 更直观的查看分组聚合结果.
# index表示: 行, columns表示: 列, values表示: 值, aggfunc表示: 聚合函数.
df.pivot_table(index='city', columns='gender_group', values='customer', aggfunc='sum')

In [ ]:
# 需求3: 按照 城市, 销售渠道(线上, 线下)划分, 分别计算 销售金额的平均值, 成本的总和. 
# 方式1: groupby() + 聚合函数.
df.groupby(['city', 'channel']).agg({'revenue':'mean', 'unit_cost':'sum'})

# 方式2: pivot_table() 透视表
df.pivot_table(index=['city', 'channel'], values=['revenue', 'unit_cost'], aggfunc={'revenue':'mean', 'unit_cost':'sum'})

# 透视表的几种变形写法.
# 变形1: 因为没有指定哪个列 mean(平均值), 哪个列sum(总和), 所以默认: 所有列都进行 平均值和总和的计算.
df.pivot_table(index=['city', 'channel'], values=['revenue', 'unit_cost'], aggfunc=['mean', 'sum'])
df.pivot_table(index=['city', 'channel'], values=['revenue', 'unit_cost'], aggfunc=['mean', 'sum']).iloc[:, [0,3]]  # 只显示需要的列

# 变形2: 透视表, 行列转置.  columns: 分组字段的值 作为列名, values: 聚合函数的结果作为值.
df.pivot_table(index='city', columns='channel', values=['revenue', 'unit_cost'], aggfunc={'revenue':'mean', 'unit_cost':'sum'})

# 3.分组过滤

In [ ]:
# 需求: 按照城市分组, 查询每组销售金额平均值.
# df.groupby('city').revenue.mean()
# df.groupby('city').revenue.agg('mean')
df.groupby('city').agg({'revenue': 'mean'})

In [ ]:
# filter()函数, 分组过滤的.
# 需求: 按照城市分组, 查询每组销售金额平均值 大于 200的全部数据. 
df.groupby('city').get_group('上海')      # 根据分组名, 获取该分组的数据. 
df.groupby('city').get_group('深圳')      # 根据分组名, 获取该分组的数据. 
df.groupby('city').get_group('广州')      # 根据分组名, 获取该分组的数据. 
df.groupby('city').get_group('北京')      # 根据分组名, 获取该分组的数据. 

# 大白话解释需求: 按照城市分组, 计算每组的销售金额的平均值, 并筛选出: 均值大于200的 所有分组的数据.
# filter(): 根据条件, 筛选出合法的数据. 
# 换换成SQL思路: select * from df where city in ('北京', '南京');
# 换换成SQL思路: select * from df where city in (select city from df group by city having avg(revenue) > 200);
df.groupby('city').filter(lambda s: s['revenue'].mean() > 200)
# df.groupby('city')['revenue'].filter(lambda s: s.mean() > 200)